In [1]:
!pip install tensorflow_io
!pip install keras-tuner
!pip install print_schema
!pip install pydub
!pip install opensmile
!pip install tqdm boto3 requests regex sentencepiece sacremoses
!pip install transformers
!pip install wandb

ERROR: Invalid requirement: '!pip'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 59.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 87.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.7/232.7 kB 32.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.1
    Uninstalling protobuf-3.20.1:
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'message_factory.py'
Consider using the `--user` option or check the permissions.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 36.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━

In [2]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc


True

In [4]:
import numpy as np
import pandas as pd
import gc

In [6]:
data = pd.read_csv("data/train.csv")
id = data.iloc[:,0].values
train_labels = data.iloc[:,1].values
train_texts = data.iloc[:,2].values

In [7]:
df = pd.read_csv("data/test.csv")
df["Content"].fillna("",inplace=True)
test_texts = list(df.loc[:,"Content"])
test_labels = list(df.loc[:,"Label"])

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit
def get_several_validation_set(content, label, val_fraction=0.2, total_splits=5, seed=0):
  sss = StratifiedShuffleSplit(n_splits=total_splits, test_size=val_fraction, random_state=seed)
  return sss.split(content, label)

In [26]:
import torch
import torchvision
import torch.nn as nn
from transformers import BertModel
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
import random
import time
import wandb

class ADdataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)
    
# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, archs,dropout,freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 256, 2

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        
        layers = []          
        layers.append(nn.Linear(D_in, archs[0]))
        layers.append(nn.BatchNorm1d(num_features=archs[0]))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout))
          
        for i in range(1,len(archs)):
          layers.append(nn.Linear(archs[i-1], archs[i]))
          layers.append(nn.BatchNorm1d(num_features=archs[i]))
          layers.append(nn.ReLU())
          layers.append(nn.Dropout(dropout))

        layers.append(nn.Linear(archs[-1], 1))
        self.classifier  = nn.Sequential(*layers)

        """
        for i in range(number_of_layers-1):
          layers.append(nn.Linear(fc_layer_size, fc_layer_size))
          layers.append(nn.BatchNorm1d(num_features=fc_layer_size))
          layers.append(nn.ReLU())
          layers.append(nn.Dropout(dropout))

        layers.append(nn.Linear(fc_layer_size, 1))
        self.classifier  = nn.Sequential(*layers)
        """

        """
        self.classifier = nn.Sequential(
            

            nn.Linear(D_in, fc_layer_size),
            nn.BatchNorm1d(num_features=fc_layer_size), ## kk
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(fc_layer_size, 1),
            #nn.BatchNorm1d(num_features=128),
            #nn.ReLU(),
            #nn.Linear(128, 1)
            
            #nn.BatchNorm1d(num_features=128), ###
            #nn.ReLU(),
            
            #nn.Dropout(0.5),

            #nn.Linear(H, D_out)
            #nn.Linear(128, 2)
            #nn.Linear(128, 1)
        )
        """
        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits
    
def initialize_model(archs,dropout,train_loader,epochs):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(archs,dropout,freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = torch.optim.Adam(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_loader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler


import random
import time

# Specify loss function
#loss_fn = nn.CrossEntropyLoss()
#loss_fn = nn.BCELoss()
loss_fn =  torch.nn.BCEWithLogitsLoss()
def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train_epoch(model, train_loader,val_loader, optim,scheduler):
      model.train()
      # For each batch of training data...
      train_loss_sum = 0
      train_accuracy_epoch = 0
      for batch in (train_loader):
          optim.zero_grad()
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['labels'].to(device)

          # Perform a forward pass. This will return logits.
          logits = model(input_ids, attention_mask)

          # Compute loss and accumulate the loss values
          logits = logits.reshape(-1) #silebilirsin
          
          loss = loss_fn(logits, labels.float())
          train_loss_sum += loss.item()

          logits_class = logits > 0.5

          train_acc = (labels == logits_class).sum().item() / labels.size(0)
          train_accuracy_epoch += train_acc

          # Perform a backward pass to calculate gradients
          loss.backward()
          # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
          # Update parameters and the learning rate
          optim.step()
          scheduler.step()
          #wandb.log({"batch loss": loss.item()})

      model.eval()
      # Tracking variables
      val_loss_sum = 0
      val_accuracy_epoch = 0
      # For each batch in our validation set...
      for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Compute logits
        with torch.no_grad():
          logits = model(input_ids, attention_mask)

          # Compute loss
          logits = logits.reshape(-1)
          loss = loss_fn(logits, labels.float())
          val_loss_sum +=loss.item()

          logits_class = logits > 0.5
          val_acc = (labels == logits_class).sum().item() / labels.size(0)
          val_accuracy_epoch += val_acc


      return model,(train_loss_sum / len(train_loader)),(train_accuracy_epoch/len(train_loader)),(val_loss_sum / len(val_loader)),(val_accuracy_epoch/len(val_loader))



def train(model, train_loader, optim, val_loader=None, epochs=50, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    train_loss_list = []
    val_loss_list = []
    train_acc_list = []
    val_acc_list = [] 
    for epoch_i in range(epochs):
        train_loss_sum = 0
        train_accuracy_epoch = 0
        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for batch in (train_loader):
            optim.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Perform a forward pass. This will return logits.
            logits = model(input_ids, attention_mask)

            # Compute loss and accumulate the loss values
            logits = logits.reshape(-1) #silebilirsin
            
            loss = loss_fn(logits, labels.float())
            train_loss_sum += loss.item()

            logits_class = logits > 0.5
            train_acc = (labels == logits_class).sum().item() / labels.size(0)
            train_accuracy_epoch += train_acc

            # Perform a backward pass to calculate gradients
            loss.backward()


            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)


            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

        avg_train_loss = np.round(train_loss_sum/len(train_loader),2)
        avg_train_acc = np.round(train_accuracy_epoch/len(train_loader),2)

        if evaluation == True:
            avg_val_loss, avg_val_acc = evaluate(model, val_loader)
        print('Epoch {}, train loss {} , val loss is {}, train acc is {}, val acc is {} '.format(epoch_i,avg_train_loss,avg_val_loss,avg_train_acc,avg_val_acc))
        train_loss_list.append(avg_train_loss)
        val_loss_list.append(avg_val_loss)
        val_acc_list.append(avg_val_acc)
        train_acc_list.append(avg_train_acc)

    print("Training complete!")
    return model,train_loss_list,val_loss_list,train_acc_list,val_acc_list


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_loss_sum = 0
    val_accuracy_epoch = 0
    # For each batch in our validation set...
    for batch in val_dataloader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      
      # Compute logits
      with torch.no_grad():
        logits = model(input_ids, attention_mask)

        # Compute loss
        logits = logits.reshape(-1)
        loss = loss_fn(logits, labels.float())
        val_loss_sum +=loss.item()
        avg_val_loss = np.round(val_loss_sum/len(val_dataloader),2)

        logits_class = logits > 0.5
        val_acc = (labels == logits_class).sum().item() / labels.size(0)
        val_accuracy_epoch += val_acc
        avg_val_acc = np.round(val_accuracy_epoch/len(val_dataloader),2)

    return avg_val_loss, avg_val_acc


def evaluate_test(model, test_dataloader):
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    test_loss_sum = 0
    test_accuracy_epoch = 0
    predictions = []
    labels_list = []
    for batch in test_dataloader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      
      # Compute logits
      with torch.no_grad():
        logits = model(input_ids, attention_mask)
        labels_list.append(labels.cpu())

        # Compute loss
        logits = logits.reshape(-1)
        loss = loss_fn(logits, labels.float())
        test_loss_sum +=loss.item()
        avg_test_loss = np.round(test_loss_sum/len(test_dataloader),5)

        logits_class = logits > 0.5
        predictions.append(logits.cpu())
        test_acc = (labels == logits_class).sum().item() / labels.size(0)
        test_accuracy_epoch += test_acc
        avg_test_acc = np.round(test_accuracy_epoch/len(test_dataloader),5)

    return avg_test_loss, avg_test_acc, predictions, labels_list


def evaluate_ensemble(models, test_dataloader):
    test_loss_sum = 0
    test_accuracy_epoch = 0
    predictions = []
    labels_list = []
    for batch in test_dataloader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      prediction = []
      for model in models:
        model.eval()
        # Compute logits
        with torch.no_grad():
          logits = model(input_ids, attention_mask)
          labels_list.append(labels.cpu())

          logits = logits.reshape(-1)
          logits_class = logits > 0.5
          prediction.append(logits_class)

      prediction_ensemble = sum(prediction) > 0.5*len(prediction)
      predictions.append(prediction_ensemble.cpu())

      test_acc = (labels == prediction_ensemble).sum().item() / labels.size(0)
      test_accuracy_epoch += test_acc
      avg_test_acc = np.round(test_accuracy_epoch/len(test_dataloader),5)

    return avg_test_acc, predictions, labels_list

In [11]:
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')

Using cache found in /home/jupyter/.cache/torch/hub/huggingface_pytorch-transformers_main


In [19]:
archs = [[512,64],[64,512],[256,512,64],[128],[128,512,256],[256,64],[256,128],[512,256],[256,64,128],[512,256,64],[128],[64,512],[256,512,128]]
batch_size = [16,16,8,8,16,16,16,16,16,16,16,16,16]
dropout = [0.4,0.5,0.3,0.3,0.4,0.3,0.4,0.5,0.3,0.4,0.5,0.3,0.3]
epochs = 15

In [16]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [28]:
from transformers import logging
logging.set_verbosity_error()

In [ ]:
for seed in range(0,100,10):
    set_seed(seed)    # Set seed for reproducibility
    seed = seed
    for i in range(len(archs)):
        index = 1
        train_and_val_splits = get_several_validation_set(train_texts, train_labels, total_splits=3, seed=seed)
        for train_index, val_index in train_and_val_splits:
            print("dr-{},arch-{},bs-{},seed-{},index-{},epoch-{}".format(dropout[i],archs[i],batch_size[i],seed,index,epochs))
            wandb.init(project="Final-ensemble", entity="miraclelinzzz")

            train_encodings = tokenizer(list(train_texts[train_index]), truncation=True, padding=True)
            val_encodings = tokenizer(list(train_texts[val_index]), truncation=True, padding=True)

            train_dataset = ADdataset(train_encodings, train_labels[train_index])
            val_dataset = ADdataset(val_encodings, train_labels[val_index])

            train_loader = DataLoader(train_dataset, batch_size=batch_size[i], shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size[i], shuffle=True)

            test_encodings = tokenizer(test_texts, truncation=True, padding=True)
            test_dataset = ADdataset(test_encodings, test_labels)
            test_loader = DataLoader(test_dataset, batch_size=batch_size[i], shuffle=False)

            wandb.run.name = "dr-{},arch-{},bs-{},seed-{},index-{},epoch-{}".format(dropout[i],archs[i],batch_size[i],seed,index,epochs)
            bert_classifier, optimizer, scheduler = initialize_model(archs[i],dropout[i],train_loader,epochs)
            
            for epoch in range(epochs):
                model, train_loss,train_acc,val_loss,val_acc = train_epoch(bert_classifier, train_loader,val_loader, optimizer,scheduler)
                #wandb.log({"loss": avg_loss, "epoch": epoch})  
                wandb.log({"Training Loss": train_loss,"Training Acc": train_acc,"Validation Loss": val_loss,"Validation Acc": val_acc})

            test_loss, test_accuracy, predictions, labels_l = evaluate_test(model, test_loader)
            wandb.summary['test_losss'] = test_loss
            wandb.summary['test_accuracy'] = test_accuracy
            if test_accuracy < 0.8:
                torch.save(bert_classifier, "weights_low/dr-{}_arch-{}_bs-{}_seed-{}_index-{}_epoch-{}.pt".format(dropout[i],archs[i],batch_size[i],seed,index,epochs))
            else:
                torch.save(bert_classifier, "weights/dr-{}_arch-{}_bs-{}_seed-{}_index-{}_epoch-{}.pt".format(dropout[i],archs[i],batch_size[i],seed,index,epochs))
            index += 1
            wandb.finish()
            gc.collect()

dr-0.4,arch-[512, 64],bs-16,seed-0,index-1,epoch-15


Training Acc,▁▂▃▅▅▆▇███████
Training Loss,█▇▆▅▅▃▃▂▂▁▁▁▁▁
Validation Acc,▁▁▃▇▇▃▄█▅█▇▇██
Validation Loss,█▇▅▄▂▅▅▄▆▁▁▂▁▁
Training Acc,1.0
Training Loss,0.15476
Validation Acc,0.91667
Validation Loss,0.30857


Training Acc,▁▂▄▄▆▇▇████████
Training Loss,█▇▆▆▄▃▂▂▁▁▁▁▁▁▁
Validation Acc,▁▁▃▇▅▇▅████▆▄▆█
Validation Loss,▅▅▅▃▅▃▃▁▁▁▁▄█▄▁
Training Acc,1.0
Training Loss,0.17262
Validation Acc,0.85417
Validation Loss,0.41119
test_accuracy,0.81786
test_losss,0.45001


dr-0.4,arch-[512, 64],bs-16,seed-0,index-2,epoch-15


Training Acc,▁▂▃▅▅▆▇████████
Training Loss,█▇▆▅▅▃▃▂▂▁▁▁▁▁▁
Validation Acc,▁▁▃▇▇▃▄█▅█▇▇███
Validation Loss,█▇▅▄▂▅▅▄▆▁▁▂▁▁▁
Training Acc,1.0
Training Loss,0.15878
Validation Acc,0.91667
Validation Loss,0.31455
test_accuracy,0.87143
test_losss,0.40477


...

dr-0.5,arch-[512, 256],bs-16,seed-60,index-2,epoch-15


Training Acc,▂▁▄▃▅▆▆█▇██████
Training Loss,██▇▇▅▅▄▃▃▂▂▁▁▁▁
Validation Acc,▃▁▃▄▅▇▆█▅▇▅▆███
Validation Loss,██▆▇▅▄▅▁█▅▅▄▂▂▂
Training Acc,1.0
Training Loss,0.09671
Validation Acc,0.85417
Validation Loss,0.37145
test_accuracy,0.75536
test_losss,0.48143


dr-0.5,arch-[512, 256],bs-16,seed-60,index-3,epoch-15


Training Acc,▁▁▁▄▄▄▇▇█▇█████
Training Loss,██▇▆▆▅▄▃▂▂▂▁▁▁▁
Validation Acc,▁▁▄▃▄▇▇▇▇▆█▅▅▇▇
Validation Loss,███▅▇▄▂▄▂▅▁▆█▁▂
Training Acc,1.0
Training Loss,0.12659
Validation Acc,0.85417
Validation Loss,0.3976
test_accuracy,0.82143
test_losss,0.48243


dr-0.3,arch-[256, 64, 128],bs-16,seed-60,index-1,epoch-15
